<a href="https://colab.research.google.com/github/smargetic/Natural_Language_Processing/blob/main/Machine_Translation/intake_preprocess_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#runtime data storage
import pandas as pd
import numpy as np

#show all rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#visualize
from IPython.display import display

#pytorch
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence

#tensorflow
import tensorflow as tf
#from tensorflow.keras.preprocessing.sequence import pad_sequence as tf_pad_sequence

#word/sentence processing
import re

#data storage
import pickle
import csv
from google.colab import files
import os
import shutil
import zipfile

In [ ]:
#import files
def import_file(fileName, sep=None):
  df = pd.read_csv(fileName, sep=sep, on_bad_lines='warn').T.reset_index().T.reset_index(drop=True)
  return df

#display file with name
def get_file_and_disp(fileName, sep=None, stringName=""):
  print("\n" + stringName + ":")

  #if zip file, unzip
  if(".zip" in fileName):
    with zipfile.ZipFile(fileName, 'r') as zip_ref:
        zip_ref.extractall("./")

    #rename for import
    fileName = fileName[:-4]

  df = import_file(fileName, sep=sep)
  display(df.head())
  return df

# #english to italian translation
# df_eng_it = get_file_and_disp(fileName="Sentence pairs in English-Italian - 2024-07-26.tsv" ,sep='\t', stringName="English to Italian")


In [ ]:
#get sample of all data - to be used for experimentation
def get_sample(df, frac=.2):
  df_sample = df.sample(frac=frac, random_state=42)
  df_sample = df_sample.reset_index(drop=True)

  return df_sample

#store sample
def dump_sample(df, fileName):
  df.to_csv(fileName, sep='\t', index=False)

# #get sample
# df_eng_it = get_sample(df_eng_it, .01)
# dump_sample(df_eng_it, "sentence_pairs_eng_it_01.tsv")

In [ ]:
#double check that there are no null versions for these sentences
def count_remove_null(df, index=1, name=""):
  #count nulls
  nulls = df[index].isnull().sum()+ df[index].eq("").sum()
  print("Nulls in " + name + ": {}".format(nulls))

  #remove nulls
  if(nulls>0):
    df.dropna(subset=[index], inplace=True)
    df = df[df[index]!=""]
    df.reset_index(drop=True, inplace=True)

  return df


# df_eng_it = count_remove_null(df_eng_it, index=1, name="English")
# df_eng_it = count_remove_null(df_eng_it, index=3, name="Italian")

In [ ]:
# #rename columns
# df_eng_it.columns = ['eng_id', 'eng_sentence', 'it_id', 'it_sentence']
# df_eng_it.head()

In [ ]:
#seperates 1-1 translations
def one_to_one_translations(df):
  df_sing = df.copy().loc[~df.duplicated(subset='eng_sentence', keep=False), :]
  df_sing = df_sing.loc[~df_sing.duplicated(subset='it_sentence',keep=False), :]
  df_sing.reset_index(drop=True, inplace=True)

  print('\nOrig size: {}'.format(df.shape[0]))
  print('Singular size: {}'.format(df_sing.shape[0]))

  return df_sing

# df_eng_it_sing = one_to_one_translations(df_eng_it)

In [ ]:
##tokenize

#get tokens
def tokenize(sentence):
  sent = re.findall(r'\b\w+\b|[^\w\s]', sentence)
  #add start of sentence and eos tokens
  sent.insert(0, '<sos>')
  sent.append('<eos>')

  return sent

#get vocabulary as dictionary - with values as indecies
def get_vocab(df, col):
  unique_tokens = list(np.unique(np.hstack(np.array(df[col]))))

  #move start of sentence and end of sentence tokens to front
  unique_tokens.insert(0, unique_tokens.pop(unique_tokens.index("<eos>")))
  unique_tokens.insert(0, unique_tokens.pop(unique_tokens.index("<sos>")))

  vocab = {k: v+2 for v, k in enumerate(unique_tokens)}

  return vocab

#encode for vocab
def encode_vocab(tokens, vocab):
    return [vocab[token] for token in tokens]

#full tokenization - returns modified pandas df, and vocabs
def tokenize_full(df, name=""):
  print('\n'+name+":")

  #split words into tokens
  df['eng_tokens'] = df['eng_sentence'].apply(tokenize)
  df['it_tokens'] = df['it_sentence'].apply(tokenize)

  #get vocab
  eng_vocab = get_vocab(df, 'eng_tokens')
  it_vocab = get_vocab(df, 'it_tokens')

  print("\tEnglish vocabulary size: {}".format(len(eng_vocab)))
  print("\tItalian vocabulary size: {}".format(len(it_vocab)))

  #encode for vocab
  df['eng_tokens_enc'] = df['eng_tokens'].apply(lambda x: encode_vocab(x, eng_vocab))
  df['it_tokens_enc'] = df['it_tokens'].apply(lambda x: encode_vocab(x, it_vocab))

  return df, eng_vocab, it_vocab

# #tokenize
# df_eng_it, eng_vocab, it_vocab = tokenize_full(df_eng_it, name="English/Italian Translations")
# df_eng_it_sing, eng_vocab_sing, it_vocab_sing = tokenize_full(df_eng_it_sing, name="English/Italian One-One Translations")


In [ ]:
### pytorch ###

#turn column of lists to column of pytorch tensors
def turn_list_col_pytorch(df, col):
  new_col = col + '_p'
  df[new_col] = df[col].apply(lambda x: torch.tensor(x))
  return df

#add padding
def pytorch_pad(df, col):
  vals = pad_sequence(df[col], batch_first=True, padding_value=0)
  df[col] = list(vals)

  return df, vals

#seperate data into batches
def pytorch_batch(eng_tens, it_tens, batch_size=64):
  dataset = TensorDataset(eng_tens, it_tens)
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

  return dataset, dataloader

#all processing to turn list columns to pytorch objects
def pytorch_preproc(df):
  #turn column of lists to column of pytorch tensors
  df = turn_list_col_pytorch(df, 'eng_tokens_enc')
  df = turn_list_col_pytorch(df, 'it_tokens_enc')

  #add padding
  df, eng_tens = pytorch_pad(df, 'eng_tokens_enc_p')
  df, it_tens = pytorch_pad(df, 'it_tokens_enc_p')

  #get batches
  dataset, dataloader = pytorch_batch(eng_tens, it_tens)

  # return df, eng_tens, it_tens, dataloader
  return df, dataloader


# # df_eng_it, eng_tens, it_tens, dataloader = pytorch_preproc(df_eng_it)
# df_eng_it, dataloader_p = pytorch_preproc(df_eng_it)
# df_eng_it.head()

In [ ]:
### tensorflow ###

#add padding
def tensorflow_pad(df, col):
  new_col = col + '_t'
  df[new_col] = list(tf.keras.preprocessing.sequence.pad_sequences(df[col].tolist(), padding='post'))
  return df

#turn list column to list of tensorflow objects
def turn_list_col_tensorflow(df, col1, col2):
  dataset = tf.data.Dataset.from_tensor_slices((df[col1].to_list(), df[col2].to_list()))
  return dataset

#padding and batch
def tensorflow_pad_batch(dataset, batch_size=64):
  padded_shapes = ([None], [None])
  padding_values = (tf.constant(0, dtype=tf.int32), tf.constant(0, dtype=tf.int32))

  dataset = dataset.padded_batch(batch_size, padded_shapes=padded_shapes, padding_values=padding_values)

  return dataset

#all processing to turn list columns to tensorflow objects
def tensorflow_preproc(df):
  #turn column of lists to column of tensorflow tensors
  df = tensorflow_pad(df, 'eng_tokens_enc')
  df = tensorflow_pad(df, 'it_tokens_enc')

  #turn list column to list of tensorflow objects
  dataset = turn_list_col_tensorflow(df, 'eng_tokens_enc_t', 'it_tokens_enc_t')

  #add padding and batch
  dataset = tensorflow_pad_batch(dataset)

  return df, dataset

# df_eng_it, dataloader_t = tensorflow_preproc(df_eng_it)
# df_eng_it.head()

In [ ]:
# # Example usage
# for input_batch, target_batch in dataset:
#     print("Input batch:", input_batch)
#     print("Target batch:", target_batch)
#     break

In [ ]:
#comprehensive function to get all data
def data_preprocessing(df_eng_it, pytorchB=True, tensorflowB=True, store=True, sample=False, download=True, sing=True):
  # #english to italian translation
  # df_eng_it = get_file_and_disp(fileName="Sentence pairs in English-Italian - 2024-07-26.tsv" ,sep='\t', stringName="English to Italian")


  #remove nulls
  df_eng_it = count_remove_null(df_eng_it, index=1, name="English")
  df_eng_it = count_remove_null(df_eng_it, index=3, name="Italian")

  #rename columns
  df_eng_it.columns = ['eng_id', 'eng_sentence', 'it_id', 'it_sentence']

  #get 1-1 translations
  df_eng_it_sing = None
  if(sing):
    df_eng_it_sing = one_to_one_translations(df_eng_it)

  #tokenize
  df_eng_it, eng_vocab, it_vocab = tokenize_full(df_eng_it, name="English/Italian Translations")
  eng_vocab_sing, it_vocab_sing = None, None
  if(sing):
    df_eng_it_sing, eng_vocab_sing, it_vocab_sing = tokenize_full(df_eng_it_sing, name="English/Italian One-One Translations")

  #turn into pytorch
  dataloader_p, dataloader_p_sing = None, None
  if(pytorchB):
    df_eng_it, dataloader_p = pytorch_preproc(df_eng_it)
    if(sing):
      df_eng_it_sing, dataloader_p_sing = pytorch_preproc(df_eng_it_sing)

    print('\nDone with Pytorch.')

  #turn into tensorflow
  dataloader_t, dataloader_t_sing = None, None
  if(tensorflowB):
    df_eng_it, dataloader_t = tensorflow_preproc(df_eng_it)
    if(sing):
      df_eng_it_sing, dataloader_t_sing = tensorflow_preproc(df_eng_it_sing)

    print('\nDone with TensorFlow.')


  #for data storage
  dataList = [df_eng_it, df_eng_it_sing, dataloader_p, dataloader_t, dataloader_p_sing, dataloader_t_sing,
                eng_vocab, it_vocab, eng_vocab_sing, it_vocab_sing]

  baseNames = ['English-Italian Dataframe ', 'English-Italian SING Dataframe ',
                  'English-Italian Pytorch Dataloader ', 'English-Italian TensorFlow Dataloader ',
                  'English-Italian Pytorch SING Dataloader ', 'English-Italian TensorFlow SING Dataloader ',
                  'English Vocabulary ', 'Italian Vocabulary ',
                  'English Vocabulary SING ', 'Italian Vocabulary SING ']


  #stored seperately b/c of ram constraints
  if(store):

    #incase we're told that this is not a sample
    sample = 100 if sample == False else int(sample*100)

    #create folder if not existant
    folderName = "English-Italian Data - " + str(sample) + "%/"
    if not os.path.exists(folderName):
        os.mkdir(folderName)

    #add in sample and folder name
    fileNames = [folderName + name + '- '+str(sample) +"%" for name in baseNames]

    #add file type
    fileNames = [name+".csv" if "Dataframe" in name
                 else name+'.pth' if "Pytorch" in name
                 else name+'.pkl' if " Vocabulary" in name
                 else name for name in fileNames]

    #add file type
    # fileNames = [name+".csv" if "Dataframe" in name el name+'.pkl' for name in baseNames]

    print('\n---------------------------------------------------------------------------')
    for i in range(0,len(fileNames)):
      #store sing only if it was set
      if((sing and ("SING" in fileNames[i])) or ("SING" not in fileNames[i])):
        print('\nSaving '+ fileNames[i] +"...")

        #csv file
        if(".csv" in fileNames[i]):
          dataList[i].to_csv(fileNames[i], index=False)
        #pytorch
        elif ('.pth' in fileNames[i]):
          if(pytorchB):
            torch.save(dataList[i], fileNames[i])
        #vocab
        elif ('.pkl' in fileNames[i]):
          with open(fileNames[i], 'wb') as f:
            pickle.dump(dataList[i], f)
        #tensorflow
        else:
          if(tensorflowB):
            tf.data.experimental.save(dataList[i], fileNames[i])

    #create zip of folder
    shutil.make_archive(folderName[:-1], 'zip', folderName)

    #download if set
    if(download):
      print('\nDownloading ' + folderName[:-1] + " Directory...")
      files.download(folderName[:-1]+'.zip')

    print('Done.')


  #dictionary of data values
  data_dict = dict(zip(baseNames, dataList))


  return data_dict


In [ ]:
#zip and download - for inidividual tsv files
def zip_download(fileName):
  #create zip of folder
  shutil.make_archive(fileName, 'zip', fileName)

  #download file
  files.download(fileName+".zip")